# Codigo preparacion

In [ ]:
!pip install "gymnasium" pygame --silent

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pygame

In [4]:
# load npy file
tree_data = np.load('paltos(3).npy', allow_pickle=True)
tree_data

array([['arbol_1', -71.04884881461739, -32.84038990910421],
       ['arbol_2', -71.04958858366999, -32.8404821794963],
       ['arbol_3', -71.04980926215778, -32.8404058633257],
       ['arbol_4', -71.0504699980531, -32.8405182859727],
       ['arbol_5', -71.04922664888424, -32.841015251972266],
       ['arbol_6', -71.04926942588176, -32.84039520943775],
       ['arbol_7', -71.05106578768262, -32.84165715069143],
       ['arbol_8', -71.05020361505561, -32.84156011856145],
       ['arbol_9', -71.05126745558296, -32.84087006184975],
       ['arbol_10', -71.04878320531743, -32.84040113716143],
       ['arbol_11', -71.04989114149105, -32.84091401967176],
       ['arbol_12', -71.04982643476353, -32.84062894259972],
       ['arbol_13', -71.0498936842084, -32.84108028071009],
       ['arbol_14', -71.04892222652431, -32.84102148268019],
       ['arbol_15', -71.05078948279294, -32.84109184771386],
       ['arbol_16', -71.04984082279348, -32.84135804156654],
       ['arbol_17', -71.0509749755355

In [5]:
import tsp_game

# Register the TravelingSalesman environment
tsp_game.register_environment()

# Create the TravelingSalesman environment
env = gym.make('TravelingSalesman-v0', tree_data=tree_data, blocked_percentage=0.2)


In [27]:
import tsp_game

# Register the TravelingSalesman environment
tsp_game.register_environment()

# Create the TravelingSalesman environment
env = gym.make('TravelingSalesman-v0', tree_data=tree_data, blocked_percentage=0.2)

# Add a running attribute to the environment
env.running = True
    
# Reset the environment
observation, info = env.reset()
    
while env.running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            env.running = False
            break
        
        if event.type == pygame.KEYDOWN:
            action = None
            if event.key == pygame.K_UP:
                action = 0
            elif event.key == pygame.K_DOWN:
                action = 1
            elif event.key == pygame.K_LEFT:
                action = 2
            elif event.key == pygame.K_RIGHT:
                action = 3
            elif event.key == pygame.K_r:  # Reset game
                observation, info = env.reset()
                continue
            elif event.key == pygame.K_q:  # Quit game
                env.running = False
                break
            
            if action is not None:
                observation, reward, done, truncated, info = env.step(action)
                env.render()
                
                if done:
                    print("Goal reached! Press 'R' to reset or 'Q' to quit")
                elif truncated:
                    print("Maximum steps reached! Press 'R' to reset or 'Q' to quit")
    
    env.render()
    pygame.time.wait(50)
    
env.close()


/home/agustin/miniconda3/envs/qlearning/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be int32, actual type: int64
  logger.warn(
/home/agustin/miniconda3/envs/qlearning/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/home/agustin/miniconda3/envs/qlearning/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be int32, actual type: int64
  logger.warn(
/home/agustin/miniconda3/envs/qlearning/lib/python3.13/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{

KeyboardInterrupt: 

In [7]:
!pip install matplotlib --quiet

In [8]:
import random
from collections import defaultdict
from IPython.display import clear_output

# Initialize Q-table
Q = defaultdict(lambda: np.zeros(env.action_space.n))

# Define parameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_decay = 0.995
min_epsilon = 0.01
num_episodes = 1

# Function to choose action based on epsilon-greedy policy
def choose_action(state):
    if random.uniform(0, 1) < epsilon:
        return env.action_space.sample()  # Explore
    else:
        return np.argmax(Q[state])  # Exploit

# Train the agent
for episode in range(num_episodes):
    state, info = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action = choose_action(tuple(state))
        next_state, reward, done, truncated, info = env.step(action)
        total_reward += reward
        
        # Update Q-value
        best_next_action = np.argmax(Q[tuple(next_state)])
        Q[tuple(state)][action] = Q[tuple(state)][action] + alpha * (reward + gamma * Q[tuple(next_state)][best_next_action] - Q[tuple(state)][action])
        
        state = next_state
    
    # Decay epsilon
    import matplotlib.pyplot as plt

    # Track rewards for plotting
    rewards = []

    # Train the agent
    for episode in range(num_episodes):
        state, info = env.reset()
        done = False
        total_reward = 0
        
        while not done:
            action = choose_action(tuple(state))
            next_state, reward, done, truncated, info = env.step(action)
            total_reward += reward
            
            # Update Q-value
            best_next_action = np.argmax(Q[tuple(next_state)])
            Q[tuple(state)][action] = Q[tuple(state)][action] + alpha * (reward + gamma * Q[tuple(next_state)][best_next_action] - Q[tuple(state)][action])
            
            state = next_state
        
        # Decay epsilon
        epsilon = max(min_epsilon, epsilon * epsilon_decay)
        
        rewards.append(total_reward)
        
        if (episode + 1) % 100 == 0:
            clear_output(wait=True)
            plt.plot(rewards)
            plt.xlabel('Episode')
            plt.ylabel('Total Reward')
            plt.title('Training Progress')
            plt.show()
            print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}")

    # Extract the best route
    state, info = env.reset()
    done = False
    best_route = [state]

    while not done:
        action = np.argmax(Q[tuple(state)])
        next_state, reward, done, truncated, info = env.step(action)
        best_route.append(next_state)
        state = next_state

    print("Best Route:", best_route)
    epsilon = max(min_epsilon, epsilon * epsilon_decay)
    
    if (episode + 1) % 100 == 0:
        print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}")

# Extract the best route
state, info = env.reset()
done = False
best_route = [state]

while not done:
    action = np.argmax(Q[tuple(state)])
    next_state, reward, done, truncated, info = env.step(action)
    best_route.append(next_state)
    state = next_state

print("Best Route:", best_route)

KeyboardInterrupt: 